# Neural Data Simulation and PCA Analysis Tutorial

This notebook demonstrates how to use the neural simulation and PCA analysis modules for neuroscience data analysis.

## Learning Objectives
1. Generate simulated neural spike trains and population activity
2. Create simulated LFP (Local Field Potential) signals
3. Prepare neural data for analysis
4. Perform Principal Component Analysis (PCA)
5. Visualize and interpret results

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np

from python_4_neuroscience.neural_simulation import (
    generate_lfp_signal,
    generate_neural_population,
    generate_spike_train,
)
from python_4_neuroscience.pca_analysis import (
    perform_pca,
    plot_pca_projection,
    plot_variance_explained,
    prepare_data_for_pca,
    reconstruct_from_pca,
)

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use("seaborn-v0_8-darkgrid")
%matplotlib inline

## Part 1: Generating Neural Spike Trains

Let's start by generating spike trains for individual neurons.

In [ ]:
# Generate a spike train for a single neuron
duration = 2.0  # seconds
firing_rate = 20.0  # Hz
dt = 0.001  # 1ms time steps

spike_train = generate_spike_train(rate=firing_rate, duration=duration, dt=dt, seed=42)

# Create time axis
time = np.arange(len(spike_train)) * dt

# Plot the spike train
plt.figure(figsize=(12, 3))
spike_times = time[spike_train == 1]
plt.vlines(spike_times, 0, 1, colors="black", linewidths=1)
plt.ylim([0, 1.2])
plt.xlabel("Time (s)")
plt.ylabel("Spike")
plt.title(f"Spike Train (firing rate = {firing_rate} Hz)")
plt.tight_layout()
plt.show()

print(f"Number of spikes: {np.sum(spike_train)}")
print(f"Average firing rate: {np.sum(spike_train) / duration:.2f} Hz")

## Part 2: Simulating Neural Population Activity

Now let's simulate activity from a population of neurons.

In [ ]:
# Generate population activity
n_neurons = 50
duration = 5.0  # seconds
base_rate = 15.0  # Hz
rate_variance = 8.0  # Hz

population_activity = generate_neural_population(
    n_neurons=n_neurons,
    duration=duration,
    base_rate=base_rate,
    rate_variance=rate_variance,
    seed=42,
)

# Plot raster plot
plt.figure(figsize=(14, 6))
for neuron_idx in range(n_neurons):
    spike_times = np.where(population_activity[neuron_idx, :] == 1)[0] * dt
    plt.vlines(spike_times, neuron_idx, neuron_idx + 1, colors="black", linewidths=0.5)

plt.xlabel("Time (s)")
plt.ylabel("Neuron Index")
plt.title(f"Population Raster Plot ({n_neurons} neurons)")
plt.xlim([0, duration])
plt.ylim([0, n_neurons])
plt.tight_layout()
plt.show()

# Calculate firing rates for each neuron
firing_rates = np.sum(population_activity, axis=1) / duration
plt.figure(figsize=(10, 4))
plt.hist(firing_rates, bins=20, edgecolor="black")
plt.xlabel("Firing Rate (Hz)")
plt.ylabel("Count")
plt.title("Distribution of Firing Rates Across Population")
plt.axvline(base_rate, color="red", linestyle="--", label=f"Mean = {base_rate} Hz")
plt.legend()
plt.tight_layout()
plt.show()

## Part 3: Simulating LFP Signals

Local Field Potentials (LFP) represent the collective electrical activity of many neurons.

In [ ]:
# Generate LFP signal with multiple frequency components
duration = 3.0  # seconds
sampling_rate = 1000.0  # Hz

# Theta (4-8 Hz), Alpha (8-12 Hz), and Gamma (30-80 Hz) rhythms
frequencies = (6.0, 10.0, 40.0)
amplitudes = (1.0, 0.7, 0.3)

time, lfp = generate_lfp_signal(
    duration=duration,
    sampling_rate=sampling_rate,
    frequencies=frequencies,
    amplitudes=amplitudes,
    noise_level=0.2,
    seed=42,
)

# Plot LFP signal
plt.figure(figsize=(14, 4))
plt.plot(time, lfp, linewidth=0.5, color="navy")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (μV)")
plt.title("Simulated LFP Signal")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot power spectrum
from scipy import signal

freqs, psd = signal.welch(lfp, fs=sampling_rate, nperseg=1024)

plt.figure(figsize=(10, 4))
plt.semilogy(freqs, psd)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power Spectral Density")
plt.title("Power Spectrum of LFP Signal")
plt.xlim([0, 60])
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Part 4: Principal Component Analysis (PCA)

PCA is a dimensionality reduction technique that finds the directions of maximum variance in the data.

In [ ]:
# Generate a larger neural population for PCA
n_neurons = 100
duration = 10.0  # seconds
dt = 0.01  # 10ms bins for smoother analysis

population_activity = generate_neural_population(
    n_neurons=n_neurons, duration=duration, base_rate=20.0, rate_variance=10.0, dt=dt, seed=42
)

print(f"Population activity shape: {population_activity.shape}")
print(f"({n_neurons} neurons × {population_activity.shape[1]} time bins)")

In [ ]:
# Prepare data for PCA
prepared_data = prepare_data_for_pca(population_activity, normalize=True)
print(f"Prepared data shape: {prepared_data.shape}")
print(f"({prepared_data.shape[0]} time bins × {prepared_data.shape[1]} neurons)")

In [ ]:
# Perform PCA
pca, transformed_data = perform_pca(prepared_data, variance_threshold=0.90)

print(f"Number of components selected: {pca.n_components_}")
print(f"Total variance explained: {np.sum(pca.explained_variance_ratio_):.2%}")
print("\nVariance explained by first 5 components:")
for i in range(min(5, pca.n_components_)):
    print(f"  PC{i+1}: {pca.explained_variance_ratio_[i]:.2%}")

In [ ]:
# Visualize variance explained
fig = plot_variance_explained(pca, figsize=(14, 5))
plt.show()

In [ ]:
# Plot 2D projection
fig = plot_pca_projection(transformed_data, pc_x=0, pc_y=1, figsize=(10, 7))
plt.show()

In [ ]:
# Plot 3D projection (PC1, PC2, PC3)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(
    transformed_data[:, 0],
    transformed_data[:, 1],
    transformed_data[:, 2],
    c=np.arange(len(transformed_data)),
    cmap="viridis",
    alpha=0.6,
    s=20,
)
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
ax.set_title("Neural Activity in 3D PC Space")
plt.tight_layout()
plt.show()

## Part 5: Reconstructing Data from PCA

We can reconstruct the original data using only a subset of principal components.

In [ ]:
# Reconstruct using different numbers of components
n_components_list = [2, 5, 10, 20]

fig, axes = plt.subplots(len(n_components_list) + 1, 1, figsize=(14, 12))

# Plot original data for one neuron
neuron_idx = 0
axes[0].plot(prepared_data[:, neuron_idx], linewidth=1, label="Original")
axes[0].set_ylabel("Normalized Activity")
axes[0].set_title(f"Original Activity (Neuron {neuron_idx})")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot reconstructions
for i, n_comp in enumerate(n_components_list):
    reconstructed = reconstruct_from_pca(pca, transformed_data, n_components=n_comp)
    axes[i + 1].plot(
        reconstructed[:, neuron_idx], linewidth=1, label=f"{n_comp} components", color="orange"
    )
    axes[i + 1].set_ylabel("Normalized Activity")
    axes[i + 1].set_title(f"Reconstruction with {n_comp} PCs")
    axes[i + 1].legend()
    axes[i + 1].grid(True, alpha=0.3)

axes[-1].set_xlabel("Time Bin")
plt.tight_layout()
plt.show()

## Summary

In this tutorial, you learned:

1. **Neural Simulation**: How to generate realistic spike trains and population activity
2. **LFP Signals**: How to create simulated local field potentials with different frequency components
3. **Data Preparation**: How to format neural data for analysis
4. **PCA**: How to perform dimensionality reduction and interpret the results
5. **Visualization**: How to visualize neural data in low-dimensional spaces
6. **Reconstruction**: How to reconstruct data from principal components

These techniques form the foundation for many neural data analysis pipelines in neuroscience!